## Using Google Places To find nearby Locations


In [28]:
from pymongo import MongoClient
import folium
import pandas as pd
import matplotlib
import numpy as np
import pandas as pd
import os
import requests
import json
from pandas.io.json import json_normalize
import re
from dotenv import load_dotenv
load_dotenv()
from SRC.formulas_calculo import*

In [29]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [30]:
"""
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }
"""
ironhack = "Paseo de la chopera 14 Madrid"
geocode(ironhack)

{'type': 'Point', 'coordinates': [-3.7011, 40.39652]}

> Using a google places Api Key to obtain the Starbucks, Vegan Restaurants, Veterinary places and Nightclubs in San Francisco.

In [31]:
import os
from dotenv import load_dotenv
import json

YOUR_API_KEY = os.getenv("YOUR_API_KEY")

url4 = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+sanfrancisco&types=store&hasNextPage=true&nextPage()=true&sensor=false&key={YOUR_API_KEY}"


In [33]:
res = requests.get(url4)
starbucks = res.json()
starbucks_near = starbucks["results"]
starbucks_near[0]

{'business_status': 'OPERATIONAL',
 'formatted_address': '3595 California St, San Francisco, CA 94118, United States',
 'geometry': {'location': {'lat': 37.786261, 'lng': -122.453194},
  'viewport': {'northeast': {'lat': 37.78754532989272,
    'lng': -122.4517972201073},
   'southwest': {'lat': 37.78484567010727, 'lng': -122.4544968798927}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/cafe-71.png',
 'id': '913b3f1febc842fd5afaf3664d0610946d19b9e1',
 'name': 'Starbucks',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1920,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112448278494642109040">Nicole Maure Briones</a>'],
   'photo_reference': 'CmRaAAAAYx5vQO5_Sc8SxeCBRd5sfblqnzfQHBQpyVsH8FHAggyDFEvoxaKpUNE_pRg3aj3vYSEEPJqsosUoljpEbUHJfdG5pPkz5Yj4FVOUDUdEYsRriN0QZNRRGnyzIpjoDrPeEhBPb4YsNDTnbT3aAISUylh3GhTNFJmzBQ7QYuCvvGP0YFoaAHNBuA',
   'width': 1080}],
 'place_id': 'ChIJgfaHTzGHhYARpAaPEsn2pV8',
 'plus_code': {'compound_code': 'QGPW+

> Making a new dictionary to obtain only the relevant information, in these case the name, latitude and longitude from the api search

In [34]:
new_dictionary_list=[]
for starbucks in starbucks_near: 
    new_dict={}
    new_dict["name"]=starbucks["name"],
    new_dict["latitude"]=starbucks["geometry"]["location"]["lat"]
    new_dict["longitude"]= starbucks['geometry']['location']['lng']
    
    new_dictionary_list.append(new_dict)

starbucks_coffe = pd.DataFrame(new_dictionary_list)
starbucks_coffe.head()

,name,latitude,longitude
0,"(Starbucks,)",37.786261,-122.453194
1,"(Starbucks,)",37.782074,-122.447080
2,"(Starbucks,)",37.789146,-122.408456
3,"(Starbucks,)",37.796647,-122.394850
4,"(Starbucks,)",37.784124,-122.407653


> Using the TransformToGeoPoint function to obtain the geopoint format later used in mongo

In [35]:
"""
def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
"""   
starbucks_coffe["geopoint"] = starbucks_coffe.apply(transformToGeoPoint, axis=1)
starbucks_coffe

,name,latitude,longitude,geopoint
0,"(Starbucks,)",37.786261,-122.453194,"{'type': 'Point', 'coordinates': [-122.453194,..."
1,"(Starbucks,)",37.782074,-122.447080,"{'type': 'Point', 'coordinates': [-122.44708, ..."
2,"(Starbucks,)",37.789146,-122.408456,"{'type': 'Point', 'coordinates': [-122.408456,..."
3,"(Starbucks,)",37.796647,-122.394850,"{'type': 'Point', 'coordinates': [-122.39485, ..."
4,"(Starbucks,)",37.784124,-122.407653,"{'type': 'Point', 'coordinates': [-122.407653,..."
5,"(Starbucks,)",37.798845,-122.449338,"{'type': 'Point', 'coordinates': [-122.4493378..."
6,"(Starbucks,)",37.777059,-122.417348,"{'type': 'Point', 'coordinates': [-122.417348,..."
7,"(Starbucks,)",37.784855,-122.399996,"{'type': 'Point', 'coordinates': [-122.3999959..."
8,"(Starbucks,)",37.739719,-122.467330,"{'type': 'Point', 'coordinates': [-122.4673297..."
9,"(Starbucks,)",37.797644,-122.430395,"{'type': 'Point', 'coordinates': [-122.430395,..."


In [36]:
starbucks_coffe.to_json("OUTPUT/starbucks_place.json",orient="records")

> Exporting the new data set to json format
> In the next lines I will repeat the same procedure but for Vegan Restaurants, Nightclubs and Veterinaries

## VEGAN RESTAURANTS


In [37]:
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.77493, -122.41942&radius=1000&type=restaurant&keyword=vegan&key={YOUR_API_KEY}"
res = requests.get(url)                                                       
vegan = res.json() 
vegan_food = vegan["results"]
len(vegan_food)

14

In [38]:
new_dictionary_list=[]
for vegan in vegan_food: 
    new_dict={}
    new_dict["name"]=vegan["name"],
    new_dict["latitude"]=vegan["geometry"]["location"]["lat"]
    new_dict["longitude"]= vegan['geometry']['location']['lng']
    
    new_dictionary_list.append(new_dict)

vegan_restaurants = pd.DataFrame(new_dictionary_list)
vegan_restaurants

,name,latitude,longitude
0,"(Ananda Fuara,)",37.777875,-122.416294
1,"(The Flying Falafel,)",37.781266,-122.411230
2,"(Loving Hut,)",37.784307,-122.406012
3,"(Thai Idea Vegetarian,)",37.783464,-122.419081
4,"(Indian Oven Restaurant,)",37.771697,-122.430536
5,"(Golden Era,)",37.781318,-122.416697
6,"(YH - BEIJING,)",37.772192,-122.430619
7,"(Shizen Vegan Sushi Bar and Izakaya,)",37.768313,-122.421682
8,"(Million Thai Restaurant,)",37.785729,-122.411463
9,"(Moya,)",37.775966,-122.414176


In [40]:
"""
def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
"""    

vegan_restaurants["geopoint"] = vegan_restaurants.apply(transformToGeoPoint, axis=1)
vegan_restaurants.head()

,name,latitude,longitude,geopoint
0,"(Ananda Fuara,)",37.777875,-122.416294,"{'type': 'Point', 'coordinates': [-122.416294,..."
1,"(The Flying Falafel,)",37.781266,-122.411230,"{'type': 'Point', 'coordinates': [-122.4112304..."
2,"(Loving Hut,)",37.784307,-122.406012,"{'type': 'Point', 'coordinates': [-122.406012,..."
3,"(Thai Idea Vegetarian,)",37.783464,-122.419081,"{'type': 'Point', 'coordinates': [-122.4190807..."
4,"(Indian Oven Restaurant,)",37.771697,-122.430536,"{'type': 'Point', 'coordinates': [-122.4305355..."


In [33]:
vegan_restaurants.to_json("OUTPUT/vegan.json",orient="records")

## NIGHTCLUBS

In [41]:

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=37.77493, -122.41942&radius=3000&type=night_club&key={YOUR_API_KEY}"
res = requests.get(url)                                                       
nightclub = res.json() 
nightclub_drunk = nightclub["results"]
nightclub_drunk[0]


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 37.7816365, 'lng': -122.402083},
  'viewport': {'northeast': {'lat': 37.7829081802915,
    'lng': -122.4006083197085},
   'southwest': {'lat': 37.7802102197085, 'lng': -122.4033062802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
 'id': 'c3a4e204f9e28725fbbac90b6a0ffa67ba4bfe22',
 'name': 'Zero Zero',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113959730907883505082">Anna Gibbs</a>'],
   'photo_reference': 'CmRaAAAA7Mn5EVo5HMR4ZnyYmJQtoo1fVfNvc_b83b3IbYhZdeeR2evXJwCRtsN94IQIpEQJ4YMrv4PNaG7HlASCK-bfWItrhYL2DXCbK-8Sw0FkOXzzV6f5tjodrV63mtTucd2rEhByNWnVfvZbZu3fEBKcOZKkGhTYi15HE5UOQ7PsGj-h-xrXr-EYaw',
   'width': 4032}],
 'place_id': 'ChIJ-VtspYCAhYAR1NlEgy5ye6o',
 'plus_code': {'compound_code': 'QHJX+M5 Yerba Buena, San Francisco, CA, USA',
  'global_code': '849VQHJX+M5'},
 'price_level':

In [42]:
new_dictionary_list=[]
for nightclub in nightclub_drunk: 
    new_dict={}
    new_dict["name"]=nightclub["name"],
    new_dict["latitude"]=nightclub["geometry"]["location"]["lat"]
    new_dict["longitude"]= nightclub['geometry']['location']['lng']
    
    new_dictionary_list.append(new_dict)

nightclub_fun = pd.DataFrame(new_dictionary_list)
nightclub_fun

,name,latitude,longitude
0,"(Zero Zero,)",37.781636,-122.402083
1,"(Smuggler's Cove,)",37.779403,-122.423372
2,"(The Valencia Room,)",37.762506,-122.421412
3,"(The Grand,)",37.779100,-122.397900
4,"(Edinburgh Castle Pub,)",37.786127,-122.418957
5,"(Top of the Mark,)",37.791749,-122.410239
6,"(The Chapel,)",37.760486,-122.421298
7,"(Madrone Art Bar,)",37.774201,-122.437375
8,"(Cat Club,)",37.775371,-122.409871
9,"(District San Francisco,)",37.778634,-122.393320


In [43]:
"""
def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
    
"""
nightclub_fun["geopoint"] = nightclub_fun.apply(transformToGeoPoint, axis=1)
nightclub_fun.head()

,name,latitude,longitude,geopoint
0,"(Zero Zero,)",37.781636,-122.402083,"{'type': 'Point', 'coordinates': [-122.402083,..."
1,"(Smuggler's Cove,)",37.779403,-122.423372,"{'type': 'Point', 'coordinates': [-122.4233722..."
2,"(The Valencia Room,)",37.762506,-122.421412,"{'type': 'Point', 'coordinates': [-122.4214116..."
3,"(The Grand,)",37.779100,-122.397900,"{'type': 'Point', 'coordinates': [-122.3979003..."
4,"(Edinburgh Castle Pub,)",37.786127,-122.418957,"{'type': 'Point', 'coordinates': [-122.4189571..."


In [38]:
nightclub_fun.to_json("OUTPUT/nightclubs.json",orient="records")

## VETERINARY

In [45]:

url= f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=veterinary+in+sanfrancisco&key={YOUR_API_KEY}"
res = requests.get(url)                                                       
veterinary = res.json() 
veterinary_dog = veterinary["results"]
len(veterinary_dog)

20

In [46]:
new_dictionary_list=[]
for veterinary in veterinary_dog: 
    new_dict={}
    new_dict["name"]=veterinary["name"],
    new_dict["latitude"]=veterinary["geometry"]["location"]["lat"]
    new_dict["longitude"]= veterinary['geometry']['location']['lng']
    
    new_dictionary_list.append(new_dict)

veterinary_mascot = pd.DataFrame(new_dictionary_list)
veterinary_mascot

,name,latitude,longitude
0,"(San Francisco Pet Hospital,)",37.776581,-122.439369
1,"(San Francisco SPCA Veterinary Hospital Mission,)",37.766408,-122.412352
2,"(Ocean Avenue Veterinary Hospital,)",37.723144,-122.453253
3,"(VCA San Francisco Veterinary Specialists,)",37.761480,-122.412279
4,(San Francisco SPCA Veterinary Hospital Pacifi...,37.791284,-122.434625
5,"(Park Animal Hospital,)",37.765639,-122.466607
6,"(San Francisco Veterinary: Stewart Alan DVM,)",37.761519,-122.412340
7,"(Arguello Pet Hospital: Heffelfinger Perry DVM,)",37.780623,-122.458587
8,"(Healthy Pets Veterinary Hospital,)",37.736881,-122.469957
9,"(Mission Pet Hospital,)",37.761288,-122.421788


In [47]:
"""
def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }
    
"""
veterinary_mascot["geopoint"] = veterinary_mascot.apply(transformToGeoPoint, axis=1)
veterinary_mascot.head()

,name,latitude,longitude,geopoint
0,"(San Francisco Pet Hospital,)",37.776581,-122.439369,"{'type': 'Point', 'coordinates': [-122.4393694..."
1,"(San Francisco SPCA Veterinary Hospital Mission,)",37.766408,-122.412352,"{'type': 'Point', 'coordinates': [-122.4123516..."
2,"(Ocean Avenue Veterinary Hospital,)",37.723144,-122.453253,"{'type': 'Point', 'coordinates': [-122.4532528..."
3,"(VCA San Francisco Veterinary Specialists,)",37.761480,-122.412279,"{'type': 'Point', 'coordinates': [-122.4122788..."
4,(San Francisco SPCA Veterinary Hospital Pacifi...,37.791284,-122.434625,"{'type': 'Point', 'coordinates': [-122.434625,..."


In [43]:
veterinary_mascot.to_json("OUTPUT/veterinary_place.json",orient="records")